In [1]:
from client import start_client
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib.pyplot import figure
from main_func import main,real_partial
from plotting import plot
from jupyterthemes import jtplot

client,w=start_client(13)
client

<Client: 'tcp://127.0.0.1:46580' processes=13 threads=13, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:46580 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 13 Cores: 13 Memory: 33.69 GB


### Main function
- make_classification parameters: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html?highlight=make_classification#sklearn.datasets.make_classification
- choose e, and chunk size
- run multiple times to produce plots


### Initialize
***

In [2]:
jtplot.style(theme='grade3') #light plot theme

batches=30000 #number of minibatches
batch_size=1 #size of each minibatch
e=0.8 #tolerance for φ(χ)

#compute samples size
training_s=batches*batch_size
n_samples=int(3*training_s/2) # 1/3 of the samples will be used for testing

#parameters for make classification
dataset_params={
    "n_samples":n_samples,
    "n_features":1000,
    "n_informative":10,
    "n_redundant":0,
    "n_repeated":0,
    "n_classes":2,
    "n_clusters_per_class":1,
    "flip_y":0,
    "weights":[0.7,0.3],
    "random_state":17
}
threshold=0.03

chunks=100
e_array=[0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
sizes=[30000,50000,100000,150000,200000,300000]
minibatches=[10,50,100]
new="no" #choose if you want to recreate the dataset

### Dataset with no noise flip_y=0
***
#### Balanced
- weights=[0.5,0,5]

In [ ]:
# 3 different e
new="no"
dataset_params["weights"]=[0.5,0.5]
Acc_threshold=[]
Rounds_threshold=[]
Time_threshold=[]
labels=[[]]
l1=[[]]
l2=[[]]

for i in e_array :
    e=i
    Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibatches[0])
    new="no"
    Acc_threshold.append(Acc)
    Rounds_threshold.append(n_rounds)
    Time_threshold.append(time_stamps)
    time.sleep(5)
E_real1,Acc_real1=real_partial(chunks*minibatches[0])
len_real1=np.array([i for i in range(len(Acc_real1))])

labels[0]="Real"

l1[0]=len_real1
l2[0]=Acc_real1

-------------------------------------------------------------

Start with num of chunks: 100 ,e: 0.05
Minibatch size: 30.0
Number of chunks for each worker:  8
In progress...
coo (1, 1, 12)
Accuracy: 88.720000
coo (2, 1, 12)
Accuracy: 89.386667
coo (3, 1, 12)
Accuracy: 89.780000
coo (4, 1, 12)
Accuracy: 89.960000
coo (5, 1, 12)
Accuracy: 90.166667
coo (6, 1, 12)
Accuracy: 90.820000
coo (7, 1, 12)
Accuracy: 91.226667
coo (8, 1, 12)
Accuracy: 91.000000
coo (9, 1, 12)
Accuracy: 91.213333
coo (10, 1, 12)
Accuracy: 91.366667
coo (11, 1, 12)
Accuracy: 91.280000
coo (12, 1, 12)
Accuracy: 91.373333
coo (13, 1, 12)
Accuracy: 91.566667
coo (14, 1, 12)
Accuracy: 91.533333
coo (15, 1, 12)
Accuracy: 91.866667
coo (16, 1, 12)
Accuracy: 91.806667
coo (17, 1, 12)
Accuracy: 91.866667
coo (18, 1, 12)
Accuracy: 92.086667
coo (19, 1, 12)
Accuracy: 92.146667
coo (20, 1, 12)
Accuracy: 91.826667
coo (21, 1, 12)
Accuracy: 92.006667
coo (22, 1, 12)
Accuracy: 91.986667
coo (23, 1, 12)
Accuracy: 91.973333
coo (2

coo (206, 1, 12)
Accuracy: 93.433333
coo (207, 1, 12)
Accuracy: 93.360000
coo (208, 1, 12)
Accuracy: 93.093333
coo (209, 1, 12)
Accuracy: 93.133333
coo (210, 1, 12)
Accuracy: 93.193333
coo (211, 1, 12)
Accuracy: 93.253333
coo (212, 1, 12)
Accuracy: 93.173333
coo (213, 1, 12)
Accuracy: 93.060000
coo (214, 1, 12)
Accuracy: 93.166667
coo (215, 1, 12)
Accuracy: 93.173333
coo (216, 1, 12)
Accuracy: 93.226667
coo (217, 1, 12)
Accuracy: 93.233333
coo (218, 1, 12)
Accuracy: 93.246667
coo (219, 1, 12)
Accuracy: 93.220000
coo (220, 1, 12)
Accuracy: 93.206667
coo (221, 1, 12)
Accuracy: 93.066667
coo (222, 1, 12)
Accuracy: 92.966667
coo (223, 1, 12)
Accuracy: 93.073333
coo (224, 1, 12)
Accuracy: 93.140000
coo (225, 1, 12)
Accuracy: 93.280000
coo (226, 1, 12)
Accuracy: 93.433333
coo (227, 1, 12)
Accuracy: 93.333333
coo (228, 1, 12)
Accuracy: 93.286667
coo (229, 1, 12)
Accuracy: 93.266667
coo (230, 1, 12)
Accuracy: 93.173333
coo (231, 1, 12)
Accuracy: 93.100000
coo (232, 2, 12)
Accuracy: 93.306667
c

coo (92, 1, 12)
Accuracy: 93.000000
coo (93, 1, 12)
Accuracy: 92.980000
coo (94, 2, 12)
Accuracy: 93.260000
coo (95, 1, 12)
Accuracy: 93.060000
coo (96, 2, 12)
Accuracy: 92.886667
coo (97, 2, 12)
Accuracy: 92.900000
coo (98, 2, 12)
Accuracy: 93.093333
coo (99, 1, 12)
Accuracy: 93.186667
coo (100, 1, 12)
Accuracy: 93.433333
coo (101, 1, 12)
Accuracy: 93.266667
coo (102, 1, 12)
Accuracy: 93.173333
coo (103, 2, 12)
Accuracy: 93.153333
coo (104, 1, 12)
Accuracy: 93.146667
coo (105, 2, 12)
Accuracy: 93.120000
coo (106, 1, 12)
Accuracy: 93.060000
coo (107, 2, 10)
Accuracy: 93.060000
coo (108, 1, 9)
Accuracy: 93.333333
coo (109, 1, 7)
Accuracy: 93.333333
coo (110, 1, 7)
Accuracy: 92.740000
coo (111, 1, 0)
Accuracy: 92.486667
End of chunks...
Coordinator: finished ...
Workers: ['finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished']
Finished  1  pass with no error...


Number of chunks for each worker:  8
I

Number of chunks for each worker:  8
In progress...
coo (11, 1, 0)
Accuracy: 94.166667
End of chunks...
Coordinator: finished ...
Workers: ['finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished']
Finished  1  pass with no error...


Number of chunks for each worker:  8
In progress...
coo (12, 1, 0)
Accuracy: 94.206667
End of chunks...
Coordinator: finished ...
Workers: ['finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished']
Finished  2  pass with no error...


Number of chunks for each worker:  8
In progress...
coo (13, 1, 0)
Accuracy: 94.406667
End of chunks...
Coordinator: finished ...
Workers: ['finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished', 'finished']
Finished  3  pass with no error...


-------------------------------------

In [ ]:
name="Plots/Threshold_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_e_"+str(e_array)+".png"
plot(l1,l2,Acc_threshold,Rounds_threshold,Time_threshold,e_array,labels,name,'threshold')

In [ ]:
# Acc_minibatch=[]
# Rounds_minibatch=[]
# Time_minibatch=[]
# e=0.2
# for i in minibatches :
#     minibatch=i
#     Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibatch)
#     new="no"
#     Acc_minibatch.append(Acc)
#     Rounds_minibatch.append(n_rounds)
#     Time_minibatch.append(time_stamps)
#     time.sleep(5)

In [ ]:
# name="Plots/Minibatch_4x_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_minib_"+str(minibatches)+".png"
# plot(l1,l2,Acc_minibatch,Rounds_minibatch,Time_minibatch,minibatches,labels,name,'minibatches')

### Save results

In [ ]:
f = open("result.txt", "a")
f.writelines(["\n\n==========================================================================="])
f.writelines(["\nNEW EXPERIMENT:\nparams:",str(dataset_params),"\nminibaches_size:"
              ,str(minibatches),"\ndifferent e's:",str(e_array),"\nn_workers: ",str(len(w))])
f.writelines(["\n___________________________________________________________________________"])
f.writelines(["\nRESULTS:\nAcc_threshold",str(Acc_threshold),"\nRounds_threshold",str(Rounds_threshold),
             "\nTime_threshold",str(Time_threshold),"\nlen_real1",str(len_real1),"\nAcc_real1",str(Acc_real1)])
f.close()

### Shutdown client

In [ ]:
client.close()
client.shutdown()